In [194]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys

import six.moves.urllib.request as request
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)

In [195]:
PATH = "C:\\spark\\CaliforniaHousing\\Py-DS-ML-Bootcamp-master\\Refactored_Py_DS_ML_Bootcamp-master\\22-Deep Learning"

In [196]:
# The CSV features in our training & test data
PATH_DATASET = PATH + os.sep + "dataset"
FILE_TRAIN = PATH_DATASET + os.sep + "iris_training.csv"
FILE_TEST = PATH_DATASET + os.sep + "iris_test.csv"

df = pd.read_csv('iris.csv')
df['target'] = df['target'].apply(int)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [197]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']
traincol = ['sepal_length','sepal_width','petal_length','petal_width']

In [198]:
#iris_col = df.drop('target',axis=1)
iris_col = df
target_col = df['target']

In [199]:
#X_train, X_test, y_train, y_test = train_test_split(iris_col, target_col, test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(iris_col, target_col, test_size=0.3)

In [200]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [201]:
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width,target
119,6.0,2.2,5.0,1.5,2
114,5.8,2.8,5.1,2.4,2
78,6.0,2.9,4.5,1.5,1
89,5.5,2.5,4.0,1.3,1
16,5.4,3.9,1.3,0.4,0


In [202]:
X_test.head()

,sepal_length,sepal_width,petal_length,petal_width,target
37,4.9,3.1,1.5,0.1,0
69,5.6,2.5,3.9,1.1,1
92,5.8,2.6,4.0,1.2,1
95,5.7,3.0,4.2,1.2,1
41,4.5,2.3,1.3,0.3,0


In [203]:
X_train.to_csv(FILE_TRAIN)

#X_train.to_csv('iris_train.csv', index=False)
#df2 = pd.read_csv('iris_training.csv', index_col='Unnamed: 0')
df2 = pd.read_csv(FILE_TRAIN)
df2.head()

,Unnamed: 0,sepal_length,sepal_width,petal_length,petal_width,target
0,119,6.0,2.2,5.0,1.5,2
1,114,5.8,2.8,5.1,2.4,2
2,78,6.0,2.9,4.5,1.5,1
3,89,5.5,2.5,4.0,1.3,1
4,16,5.4,3.9,1.3,0.4,0


In [204]:
X_test.to_csv(FILE_TEST)
X_test.head()

,sepal_length,sepal_width,petal_length,petal_width,target
37,4.9,3.1,1.5,0.1,0
69,5.6,2.5,3.9,1.1,1
92,5.8,2.6,4.0,1.2,1
95,5.7,3.0,4.2,1.2,1
41,4.5,2.3,1.3,0.3,0


In [205]:
feature_names = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth']

print(FILE_TRAIN)

C:\spark\CaliforniaHousing\Py-DS-ML-Bootcamp-master\Refactored_Py_DS_ML_Bootcamp-master\22-Deep Learning\dataset\iris_training.csv


In [206]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API

def my_input_fn(file_path, repeat_count=1, shuffle_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0], [0.], [0.], [0.], [0.], [0]])
        #parsed_line = tf.decode_csv(line, [[0], [0.], [0.], [0.], [0.]])
        label = parsed_line[-1]  # Last element is the label
        del parsed_line[-1]  # Delete last element
        del parsed_line[0]  # Delete first index element due to csv convert
        features = parsed_line  # Everything but last elements are the features
        d = dict(zip(feature_names, features)), label
        return d
    
    dataset = (tf.data.TextLineDataset(file_path)  # Read text file
        .skip(1)  # Skip header row
        .map(decode_csv, num_parallel_calls=4)  # Decode each line
        .cache() # Warning: Caches entire dataset, can cause out of memory
        .shuffle(shuffle_count)  # Randomize elems (1 == no operation)
        .repeat(repeat_count)    # Repeats dataset this # times
        .batch(64)
        .prefetch(1)  # Make sure you always have 1 batch ready to serve
        )
    
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    print(batch_features)
    print(batch_labels)
    return batch_features, batch_labels

In [207]:
def my_model_fn(
    features, # This is batch_features from input_fn
    labels,   # This is batch_labels from input_fn
    mode):    # And instance of tf.estimator.ModeKeys, see below

    if mode == tf.estimator.ModeKeys.PREDICT:
        tf.logging.info("my_model_fn: PREDICT, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.EVAL:
        tf.logging.info("my_model_fn: EVAL, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info("my_model_fn: TRAIN, {}".format(mode))

    # All our inputs are feature columns of type numeric_column

    feature_columns = [
        tf.feature_column.numeric_column(feature_names[0]),
        tf.feature_column.numeric_column(feature_names[1]),
        tf.feature_column.numeric_column(feature_names[2]),
        tf.feature_column.numeric_column(feature_names[3])
    ]

    # Create the layer of input
    input_layer = tf.feature_column.input_layer(features, feature_columns)

    # Definition of hidden layer: h1
    # We implement it as a fully-connected layer (tf.layers.dense)
    # Has 10 neurons, and uses ReLU as the activation function
    # Takes input_layer as input

    h1 = tf.layers.Dense(10, activation=tf.nn.relu)(input_layer)

    # Definition of hidden layer: h2 (this is the logits layer)
    # Similar to h1, but takes h1 as input

    h2 = tf.layers.Dense(10, activation=tf.nn.relu)(h1)

    # Output 'logits' layer is three number = probability distribution
    # between Iris Setosa, Versicolor, and Viginica

    logits = tf.layers.Dense(3)(h2)

    # class_ids will be the model prediction for the class (Iris flower type)
    # The output node with the highest value is our prediction

    predictions = { 'class_ids': tf.argmax(input=logits, axis=1) }

    # 1. Prediction mode
    # Return our prediction

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Evaluation and Training mode
    # Calculate the loss

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Calculate the accuracy between the true labels, and our predictions

    accuracy = tf.metrics.accuracy(labels, predictions['class_ids'])

    # 2. Evaluation mode
    # Return our loss (which is used to evaluate our model)
    # Set the TensorBoard scalar my_accurace to the accuracy
    # Obs: This function only sets value during mode == ModeKeys.EVAL
    # To set values during training, see tf.summary.scalar

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            eval_metric_ops={'my_accuracy': accuracy})

    # If mode is not PREDICT nor EVAL, then we must be in TRAIN

    assert mode == tf.estimator.ModeKeys.TRAIN, "TRAIN is only ModeKey left"

    # 3. Training mode

    # Default optimizer for DNNClassifier: Adagrad with learning rate=0.05
    # Our objective (train_op) is to minimize loss
    # Provide global step counter (used to count gradient updates)

    optimizer = tf.train.AdagradOptimizer(0.05)
    train_op = optimizer.minimize(
        loss,
        global_step=tf.train.get_global_step())

    # Set the TensorBoard scalar my_accuracy to the accuracy
    # Obs: This function only sets the value during mode == ModeKeys.TRAIN
    # To set values during evaluation, see eval_metrics_ops

    tf.summary.scalar('my_accuracy', accuracy[1])
    # Return training operations: loss and train_op
    return tf.estimator.EstimatorSpec(
        mode,
        loss=loss,
        train_op=train_op)

In [208]:
# Create a custom estimator using my_model_fn to define the model
tf.logging.info("Before classifier construction")

classifier = tf.estimator.Estimator(
    model_fn=my_model_fn,
    model_dir=PATH)  # Path to where checkpoints etc are stored

tf.logging.info("...done constructing classifier")

INFO:tensorflow:Before classifier construction
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\spark\\CaliforniaHousing\\Py-DS-ML-Bootcamp-master\\Refactored_Py_DS_ML_Bootcamp-master\\22-Deep Learning', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021DAFD94160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 

In [209]:

# 500 epochs = 500 * 120 records [60000] = (500 * 120) / 32 batches = 1875 batches
# 4 epochs = 4 * 30 records = (4 * 30) / 32 batches = 3.75 batches

# Train our model, use the previously function my_input_fn
# Input to training is a file with training example
# Stop training after 8 iterations of train data (epochs)

tf.logging.info("Before classifier.train")

classifier.train(
    input_fn=lambda: my_input_fn(FILE_TRAIN, 500, 256))

tf.logging.info("...done classifier.train")



# Evaluate our model using the examples contained in FILE_TEST
# Return value will contain evaluation_metrics such as: loss & average_loss

tf.logging.info("Before classifier.evaluate")

evaluate_result = classifier.evaluate(
    input_fn=lambda: my_input_fn(FILE_TEST, 4))

tf.logging.info("...done classifier.evaluate")

tf.logging.info("Evaluation results")
for key in evaluate_result:
    tf.logging.info("   {}, was: {}".format(key, evaluate_result[key]))

INFO:tensorflow:Before classifier.train
{'SepalLength': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float32>, 'SepalWidth': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=float32>, 'PetalLength': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float32>, 'PetalWidth': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float32>}
Tensor("IteratorGetNext:4", shape=(?,), dtype=int32, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: TRAIN, train
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\spark\CaliforniaHousing\Py-DS-ML-Bootcamp-master\Refactored_Py_DS_ML_Bootcamp-master\22-Deep Learning\model.ckpt-10675
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10675 into C:\spark\CaliforniaHousing\Py-DS-ML-Bootcamp-master\Refactored_Py_DS_ML_Bootcamp-master\22-Deep Learn

In [210]:
# Predict the type of some Iris flowers.
# Let's predict the examples in FILE_TEST, repeat only once.

predict_results = classifier.predict(
    input_fn=lambda: my_input_fn(FILE_TEST, 1))

In [211]:
#predict_results[0]

In [212]:
tf.logging.info("Prediction on test file")
final_preds  = []
for pred in predict_results:
    final_preds.append(pred["class_ids"])
#i=0
#for prediction in predict_results:
    # Will print the predicted class, i.e: 0, 1, or 2 if the prediction
    # is Iris Setosa, Vericolor, Virginica, respectively.
    #tf.logging.info("...{}".format(prediction["class_ids"]))
#    print(prediction["class_ids"])
    #pred_ser[i] = prediction["class_ids"]
    #i=i+1

INFO:tensorflow:Prediction on test file
{'SepalLength': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float32>, 'SepalWidth': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=float32>, 'PetalLength': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float32>, 'PetalWidth': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float32>}
Tensor("IteratorGetNext:4", shape=(?,), dtype=int32, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: PREDICT, infer
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\spark\CaliforniaHousing\Py-DS-ML-Bootcamp-master\Refactored_Py_DS_ML_Bootcamp-master\22-Deep Learning\model.ckpt-11496
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [213]:
np.shape(final_preds)

(45,)

In [214]:
from sklearn.metrics import classification_report,confusion_matrix

In [215]:
#print(confusion_matrix(y_test,predict_results))
type(y_test)

pandas.core.series.Series

In [216]:
print(confusion_matrix(y_test,final_preds))

[[17  0  0]
 [ 0 14  0]
 [ 0  0 14]]


In [217]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        14

   micro avg       1.00      1.00      1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

